# Concert halls in Switzerland : a territorial study

## Introduction

Culture is a major factor when measuring a city's attractivity, whether it is for attracting tourists, skilled people, or simply to increase the well being
of the inhabitants and their quality of living.

There are regular discussions about cultural fundings, and here in Switzerland, the language situation (with 4 different official languages, but we will probably not have 'Romanche' speaking cities, given it is a rural population, and represents less than one % to the total population), and the federal regime, with highly independant states (called canton), makes this issue even more closely followed.

One of the component of cultural activity, is the music scene, and one way to measure how lively is the musical scene in a city, is through the concert halls.

The following study, aims to compare the major cities in Swizerland in terms of concert halls per capita, to provide an insight to the federal authorities, music professionals or simply to people interested in the cultural state of things from a numbers perspective, on the current distribution of the venues between the cities, opening the door for more focused studies on the budget side of things, to see how the discrepancies, if any, between the different territories could be explained, solved.

By grouping the cities into distinct clusers, based on the number of venues per capita, but also the number of inhabitants, we should be able to provide the insight needed to explore further the issue.
Through a chloropeth map, we will also be able to see how the territory part, affects the distribution.

## Data sources

### I. Data on Cities 

Data on cities will include:

- Geospatial coordinates
- The state in which the city is located
- The population
- The name of the city

All this data will be scrapped from the website https://simplemaps.com/data/ch-cities as a downloadable csv.



### II. Data on concert halls

Based on foursquare, we will be able to retrieve the venues, located in the different cities.
Here is an example of concert hall as described in foursquare.
https://foursquare.com/v/lusine/4adcdab5f964a5209a5021e3
A concert hall, is considered a 'Category' and this data should be easy to extract using the API.

We will be only focusing on the number of venues, we will need to defines the radius used to include a venue in the city or not.


# Data processing

## I. Get the data on cities


In [13]:
#Import and install all the needed libraries

%pip install geopy
%pip install requests
!pip install folium

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import requests # library to handle requests
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# The code was removed by Watson Studio for sharing.

,city,lat,lng,country,iso2,admin_name,capital,population,population_proper
0,Zürich,47.3786,8.5400,Switzerland,CH,Zürich,admin,434008.0,434008.0
1,Geneva,46.2000,6.1500,Switzerland,CH,Genève,admin,201818.0,201818.0
2,Basel,47.5606,7.5906,Switzerland,CH,Basel-Stadt,admin,177595.0,177595.0
3,Lausanne,46.5333,6.6333,Switzerland,CH,Vaud,admin,138905.0,138905.0
4,Bern,46.9480,7.4474,Switzerland,CH,Bern,primary,133798.0,133798.0


In [4]:
#let's drop all unwanted columns : iso2 / country / capital /population

cities_data = df_data_1.drop(['iso2', 'country','capital','population'], axis=1)
cities_data.head()

,city,lat,lng,admin_name,population_proper
0,Zürich,47.3786,8.5400,Zürich,434008.0
1,Geneva,46.2000,6.1500,Genève,201818.0
2,Basel,47.5606,7.5906,Basel-Stadt,177595.0
3,Lausanne,46.5333,6.6333,Vaud,138905.0
4,Bern,46.9480,7.4474,Bern,133798.0


## II. Get the foursquare data

### Before gathering the foursquare data, let's first have a look of a map of Switzerland major cities

In [142]:
address = 'Switzerland, CH'

geolocator = Nominatim(user_agent="ch_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Switzerland are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Switzerland are 46.7985624, 8.2319736.


In [143]:
import folium

# create map of Toronto using latitude and longitude values
map_ch = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, city in zip(cities_data['lat'], cities_data['lng'],cities_data['city']):
    label = '{}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ch)  
    
map_ch

In [198]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: 1WPGJJATDG0S552UF1E24XIIZF43FC3UY15LLPYKQCYOYNUP
CLIENT_SECRET:VUY2T11UDN0MIUHCSQP2BWEQ2YAHDOR5HIRXITPSJFHJFTOX


In [199]:
LIMIT = 200 # limit of number of venues returned by Foursquare API

radius = 5000 # define radius


url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1WPGJJATDG0S552UF1E24XIIZF43FC3UY15LLPYKQCYOYNUP&client_secret=VUY2T11UDN0MIUHCSQP2BWEQ2YAHDOR5HIRXITPSJFHJFTOX&v=20180604&ll=46.7985624,8.2319736&radius=5000&limit=200'

In [200]:
results = requests.get(url).json()
#results

print(results)

{'meta': {'code': 429, 'errorType': 'quota_exceeded', 'errorDetail': 'Quota exceeded', 'requestId': '600d5bc83cdc3e1394648720'}, 'response': {}}


In [25]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [171]:
venues = results['response']['groups'][0]['items']

print(venues)

nearby_venues = json_normalize(venues) # flatten JSON

print(nearby_venues)

[{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '4c596464d3aee21e76026855', 'name': 'Melchsee-Frutt', 'location': {'lat': 46.77555299796782, 'lng': 8.268671035766602, 'labeledLatLngs': [{'label': 'display', 'lat': 46.77555299796782, 'lng': 8.268671035766602}], 'distance': 3792, 'cc': 'CH', 'city': 'Melchsee-Frutt', 'state': 'Obwalden', 'country': 'Schweiz', 'formattedAddress': ['Melchsee-Frutt', 'Schweiz']}, 'categories': [{'id': '4bf58dd8d48988d1e9941735', 'name': 'Ski Area', 'pluralName': 'Ski Areas', 'shortName': 'Ski Area', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/ski_snowboard_', 'suffix': '.png'}, 'primary': True}], 'photos': {'count': 0, 'groups': []}}, 'referralId': 'e-0-4c596464d3aee21e76026855-0'}, {'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'i

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


In [187]:
LIMIT = 200

In [188]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['city', 
                  'lat', 
                  'lng', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [190]:
ch_venues = getNearbyVenues(names=cities_data['city'],
                                latitudes=cities_data['lat'],
                                longitudes=cities_data['lng']
                                )

Zürich


KeyError: 'groups'

In [197]:
print(ch_venues.shape)

(6057, 7)


In [192]:
ch_test = ch_venues.groupby('Venue Category').count()
ch_test_distinct = ch_venues['Venue Category'].unique()
print(ch_test_distinct)

['Chocolate Shop' 'Plaza' 'Gourmet Shop' 'Bar'
 'Vegetarian / Vegan Restaurant' 'Sporting Goods Shop' 'Pedestrian Plaza'
 'Cocktail Bar' 'Sandwich Place' 'Hotel Bar' 'Watch Shop' 'Hotel' 'Café'
 'Falafel Restaurant' 'Cupcake Shop' 'Liquor Store' 'Asian Restaurant'
 'Bakery' 'Train Station' 'Snack Place' 'Department Store' 'Shoe Store'
 'Movie Theater' 'Gym / Fitness Center' 'Swiss Restaurant' 'Dessert Shop'
 'Furniture / Home Store' 'Cultural Center' 'Juice Bar' 'Gastropub'
 'Japanese Restaurant' 'Art Museum' 'Coffee Shop' 'Bookstore'
 'Burger Joint' 'Cuban Restaurant' 'Italian Restaurant' 'Sushi Restaurant'
 'Pool' 'Electronics Store' 'Supermarket' 'Science Museum' 'Diner'
 'Thai Restaurant' 'Flea Market' 'Spanish Restaurant' 'French Restaurant'
 'Waterfront' 'Arts & Crafts Store' 'Trade School' 'Park' 'Wine Bar'
 'Mountain' 'Ice Cream Shop' 'Opera House' 'Bratwurst Joint'
 'Neighborhood' 'Steakhouse' 'Boutique' 'Jewelry Store' 'Art Gallery'
 'Fountain' 'History Museum' 'Peruvian Rest

### We can see unfortunately, that our first asumption, that the concert halls could be retrieved using the category 'Concert Hall' is false, we will need to add more categories, as the data inputed by the users includes different categories (for example 'Nightclub'), this extension will probably lead to some false positives, we will see how we can handle those later.

In [193]:
ch_concert_test = ch_venues[ch_venues['Venue Category'].isin(['Concert Hall','Music Venue','Opera House','Performing Arts Venue','Nightclub'])].reset_index(drop=True)
#ch_concert_halls

In [194]:
ch_concert_test.head()
print(ch_concert_test.shape)

(74, 7)


In [195]:
ch_concert_test.head(75)

,city,lat,lng,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Zürich,47.3786,8.5400,Opernhaus,47.365212,8.546604,Opera House
1,Geneva,46.2000,6.1500,Victoria Hall,46.201467,6.141059,Concert Hall
2,Geneva,46.2000,6.1500,Les Enfants Terribles,46.193950,6.145032,Music Venue
3,Lausanne,46.5333,6.6333,Les Docks,46.522425,6.619011,Nightclub
4,Bern,46.9480,7.4474,Rendez-vous Bundesplatz,46.947162,7.444121,Performing Arts Venue
...,...,...,...,...,...,...,...
69,Hinwil,47.3033,8.8444,Rock the Ring,47.289021,8.817097,Concert Hall
70,Hinwil,47.3033,8.8444,Kamäleon,47.268352,8.871696,Nightclub
71,Davos,46.8091,9.8398,Hotel Grischa - Pulsa Bar & Lounge,46.791510,9.820268,Nightclub
72,Aarberg,47.0438,7.2736,KuFa Lyss,47.080595,7.294434,Music Venue


### By looking at the data, we can see that we have several issues:

i. Duplicates, related to the 'radius' parameter used in our foursquare calls
ii. Inclusions in the 'city' columns, of neighborhoods, rather than cities, specially for the main cities (Zurich, Geneva and Bern)

Given that the result set is small, we will try to clean it up manually before going further

In [196]:
#first let's remove the duplicates
ch_cleaned1 = ch_concert_test.drop_duplicates(subset=['Venue'], keep='last').reset_index(drop=True)
ch_cleaned1.head(100)
print(ch_cleaned1.shape)

ch_cleaned1.head(50)

(49, 7)


,city,lat,lng,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Zürich,47.3786,8.5400,Opernhaus,47.365212,8.546604,Opera House
1,Winterthur,47.4992,8.7267,Albani Music Club,47.498395,8.729155,Nightclub
2,Winterthur,47.4992,8.7267,Salzhaus,47.497550,8.722566,Nightclub
3,Sankt Gallen,47.4233,9.3772,Rock Story,47.427014,9.374442,Music Venue
4,Sankt Gallen,47.4233,9.3772,Palace,47.426564,9.373650,Music Venue
5,Sankt Gallen,47.4233,9.3772,Lokremise,47.421551,9.365820,Performing Arts Venue
6,Lugano,46.0103,8.9625,Living Room Lugano,46.020717,8.958074,Nightclub
7,Biel/Bienne,47.1372,7.2472,Gaskessel | Coupole,47.135980,7.250273,Concert Hall
8,Thun,46.7590,7.6300,KKThun,46.742852,7.629493,Concert Hall
9,Neuchâtel,46.9903,6.9306,Case à Chocs,46.989271,6.921655,Music Venue


### Let's clean out the data, in order to avoid cities belonging to a bigger aglomeration in our table, once again we will be using data from
https://simplemaps.com/data/ch-cities

####  'Geneva'

'Onex'


#### 'Lausanne'

Pully

#### 'Bern'

Ostermundigen

#### 'Basel'
'Riehen'
Muttenz
Pratteln

#### Zurich

'Kloten'
'Adliswil'
Regensdorf

In [208]:
ch_cleaned = ch_cleaned1.replace('Onex','Geneva').reset_index(drop=True)

ch_cleaned.head(50)

,city,lat,lng,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Zürich,47.3786,8.5400,Opernhaus,47.365212,8.546604,Opera House
1,Winterthur,47.4992,8.7267,Albani Music Club,47.498395,8.729155,Nightclub
2,Winterthur,47.4992,8.7267,Salzhaus,47.497550,8.722566,Nightclub
3,Sankt Gallen,47.4233,9.3772,Rock Story,47.427014,9.374442,Music Venue
4,Sankt Gallen,47.4233,9.3772,Palace,47.426564,9.373650,Music Venue
5,Sankt Gallen,47.4233,9.3772,Lokremise,47.421551,9.365820,Performing Arts Venue
6,Lugano,46.0103,8.9625,Living Room Lugano,46.020717,8.958074,Nightclub
7,Biel/Bienne,47.1372,7.2472,Gaskessel | Coupole,47.135980,7.250273,Concert Hall
8,Thun,46.7590,7.6300,KKThun,46.742852,7.629493,Concert Hall
9,Neuchâtel,46.9903,6.9306,Case à Chocs,46.989271,6.921655,Music Venue


## Now let's examine the results for Geneva, just to assess how reliable is the data, we have various way of doing so, but we will compare the foursqure results with data from a cultural agenda.
https://www.leprogramme.ch/concerts/Geneve

In [151]:

ch_geneva = ch_concert_test[ch_concert_test['city'] == 'Geneva']


ch_test2 = ch_zurich.groupby('Venue Category').count()
ch_test2_distinct = ch_zurich['Venue Category'].unique()
print(ch_test2_distinct)
ch_geneva.head(100)

['Neighborhood' 'Plaza' 'Dessert Shop' 'Steakhouse' 'Park' 'Boutique'
 'Jewelry Store' 'Swiss Restaurant' 'Art Museum' 'Hotel' 'Art Gallery'
 'Ice Cream Shop' 'Bookstore' 'Waterfront' 'French Restaurant'
 'Movie Theater' 'Department Store' 'Italian Restaurant' 'Fountain'
 'History Museum' 'Peruvian Restaurant' 'Concert Hall' 'Café'
 'Coworking Space' 'Farmers Market' 'Coffee Shop' 'Music Venue'
 'Burger Joint' 'Cocktail Bar' 'Japanese Restaurant' 'Beach'
 'Chocolate Shop' 'Bar' 'Tea Room' 'Accessories Store'
 'Falafel Restaurant' 'Chinese Restaurant' 'Vietnamese Restaurant'
 'Electronics Store' 'Dim Sum Restaurant' 'Thai Restaurant' 'Supermarket'
 'Bistro' 'Salon / Barbershop' 'Restaurant' 'Museum' 'Noodle House']


,city,lat,lng,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Geneva,46.2,6.15,Victoria Hall,46.201467,6.141059,Concert Hall
2,Geneva,46.2,6.15,Les Enfants Terribles,46.193950,6.145032,Music Venue


## Foursqure limits

Duplicates
Cannot be sure that is complete
let's compare with another source for Geneva concerts place
Festivals
limits linked with the subscription type